# Python OOP Fundamentals

This notebook teaches Object-Oriented Programming (OOP) basics in Python by **building reusable classes** step-by-step.

By the end, we'll have a working mini card game using these classes:
- `Card`
- `Deck`
- `Hand`
- `Player`
- `Game`

Along the way we’ll learn:
- classes and objects
- `__init__` and attributes
- methods
- encapsulation (keeping logic inside the right class)
- composition (objects containing other objects)
- simple inheritance

## Why OOP?

OOP helps us organize code by bundling:
- **data** (attributes / state)
- **behavior** (methods)

Instead of scattered variables and functions, we model real concepts:
- A `Card` has a `rank` and `suit`
- A `Deck` *contains* many `Card`s and can `shuffle()` and `draw()`
- A `Player` has a `Hand` and can `play_card()`


In [2]:
# This is your class
class Austin:
    # This is a method
    def say_hello(self):
        print("Hello!")

# This is an object
a = Austin()
a.say_hello()

a2 = Austin()
a2.say_hello()


Hello!
Hello!


In [5]:
class Car:
    def drive(self):
        print("Vroom Vroom") 

tesla = Car()
mercedes = Car()

tesla.drive()

Vroom Vroom


## `__init__`, `self`, and attributes

- `__init__` runs when you create a new object.
- `self` refers to "this specific object."
- Attributes belong to the object (each object gets its own copy).


In [6]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank # attribute
        self.suit = suit # attribute

    def describe(self):
        return f"{self.rank} of {self.suit}"
    
c1 = Card("A", "Spades")
c2 = Card("10", "Hearts")

c1.describe()

'A of Spades'

The following is incorrect, since there is no 'self' in front of the attributes:

In [ ]:
class Dog:
    def __init__(breed, age):
        breed = breed
        age = age


## Encapsulation

Instead of writing logic *outside* the object…

Bad style:
```python
# scattered logic
if card.rank == "A": ...
```

Better OOP style: put logic inside the class and make the class responsible for its own behavior

Next: Teach `Card` how to compute its own value.

In [ ]:
class Card:
    RANK_VALUES = {
        "2": 2, "3": 3, "4": 4, "5": 5, "6": 6,
        "7": 7, "8": 8, "9": 9, "10": 10,
        "J": 11, "Q": 12, "K": 13, "A": 14
    }

    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def describe(self):
        return f"{self.rank} of {self.suit}"
    
    def value(self):
        return Card.RANK_VALUES[self.rank]
    
c = Card("K", "Diamonds")
print(c.describe(), "| value =", c.value())

K of Diamonds  | value = 13


In [18]:
c1 = Card("K", "Diamonds")
c2 = Card("A", "Spades")

c2.value() > c1.value()

True

## Composition (building bigger systems from smaller classes)

Now we’ll make a `Deck` that contains many `Card` objects.

- a `Deck` is not just data; it has behavior (`shuffle`, `draw`)
- it manages its internal cards

In [32]:
import random

class Deck:
    SUITS = ["Hearts", "Diamonds", "Clubs", "Spades"]
    RANKS = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]

    def __init__(self):
        self._cards = [] # underscore hints: "internal use"
        self.reset()

    def reset(self):
        self._cards = [Card(rank, suit) for suit in Deck.SUITS for rank in Deck.RANKS]

    def shuffle(self):
        random.shuffle(self._cards)

    def draw(self):
        if not self._cards:
            return None
        return self._cards.pop()
    
    def cards_left(self):
        return len(self._cards)
    
deck = Deck()
deck.shuffle()

print(deck.draw().describe())
print("Cards left:", deck.cards_left())

2 of Diamonds
Cards left: 51


## Hand

A `Hand` represents the cards a player holds.
It should handle things like:
- adding cards
- playing a card
- showing contents
- computing strongest card, etc.

Notice how we reuse `Card` again.


In [48]:
class Hand:
    def __init__(self):
        self._cards = []
    
    def add(self, card):
        if card is not None:
            self._cards.append(card)

    def size(self):
        return len(self._cards)
    
    def show(self):
        """Return pretty strings for display."""
        return [c.describe() for c in self._cards]
    
    def get_cards(self):
        """Expose cards as a read-only copy for decision making"""
        return list(self._cards)
    
    def play_index(self, idx):
        """Remove and return the card at position idx"""
        if idx < 0 or idx >= len(self._cards):
            return None
        return self._cards.pop(idx)
    
h = Hand()
h.add(Card("A", "Spades"))
h.add(Card("K", "Diamonds"))
print("Hand:", h.show())
print(h.get_cards())
print("Played:", h.play_index(0).describe())
print("Hand now:", h.show())

Hand: ['A of Spades', 'K of Diamonds']
[<__main__.Card object at 0x10bf9b6b0>, <__main__.Card object at 0x1120e5c10>]
Played: A of Spades
Hand now: ['K of Diamonds']


In [58]:
h = Hand()

h.add(Card("A", "Hearts"))
h.add(Card("K", "Spades"))
h.add(Card("Q", "Diamonds"))
h.add(Card("J", "Clubs"))
print(h.show())

print(h.get_cards()[0].value())

['A of Hearts', 'K of Spades', 'Q of Diamonds', 'J of Clubs']
14
